In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense,Activation
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
%matplotlib inline

Using TensorFlow backend.


In [23]:
def arima_ann(column_list):
    column_list = column_list.tolist()
    training_len = int(len(column_list)*0.7)
    train,test = column_list[:training_len] , column_list[training_len:]
    training_data = train
    error_list = []
    prediction_list = []
    for t in range(len(test)):
        model = ARIMA(training_data, order=(7,0,0))
        model_fit = model.fit(disp=0)
        predicted_value = model_fit.forecast()
        predicted_value = predicted_value[0]
        error_list.append(test[t] - predicted_value)
        prediction_list.append(predicted_value)
        obs = test[t]
        training_data.append(obs)
#         print('predicted=%f, expected=%f error=%f' % (predicted_value, obs,abs(predicted_value-obs)))
    error = mean_squared_error(test, prediction_list)
    print('Test MSE: %.3f' % error)
    model = ARIMA(train, order=(7,0,0))
    model_fit = model.fit(disp=0)
    training_error = model_fit.resid
    
    train_X,train_Y = [],[]
    for i in range(0 , len(training_error) - 7):
        train_X.append(training_error[i:i+7])
        train_Y.append(training_error[i+7])
    
    new_train_X,new_train_Y = [],[]
    for i in train_X:
        new_train_X.append(i.reshape(-1))
    for i in train_Y:
        new_train_Y.append(i.reshape(-1))
    
    new_train_X = np.array(new_train_X)
    new_train_Y = np.array(new_train_Y)
    
    model = Sequential()
    model.add(Dense(100 , activation="tanh" , init="uniform" , input_dim=7))
    model.add(Dense(50 , activation="tanh" , init="uniform"))
    model.add(Dense(25 , activation="tanh" , init="uniform"))
    model.add(Dense(1 , activation="linear"))
    model.compile("Adam", loss="mean_squared_error")
    model.fit(new_train_X , new_train_Y , batch_size=20, epochs=400,verbose=0)
    test_data = []
    for i in error_list:
        try:
            test_data.append(i[0])
        except:
            test_data.append(i)
    
    test_data = np.array(test_data)
    
    test_X,test_Y = [],[]
    for i in range(0 , len(test_data) - 7):
        test_X.append(test_data[i:i+7])
        test_Y.append(test_data[i+7])

    
    new_test_X,new_test_Y = [],[]
    
    for i in test_X:
        new_test_X.append(i.reshape(-1))
    for i in test_Y:
        new_test_Y.append(i.reshape(-1))
    
    new_test_X = np.array(new_test_X)
    new_test_Y = np.array(new_test_Y)

    predict_error = model.predict(new_test_X)
    arima_ann_predict = prediction_list[7:]+predict_error
    print("error=",mean_squared_error(test[7:] ,arima_ann_predict))
    return arima_ann_predict



In [24]:
data=pd.read_csv("cleaned_Data.csv")
data.set_index("date" , inplace = True)
data.index = pd.DatetimeIndex(data.index)
aggregate_data = data.pivot_table(index = data.index.date)
aggregate_data.drop(["rain"] , axis=1 , inplace=True)

In [25]:
new_data = pd.DataFrame()

In [12]:
pred = arima_ann(aggregate_data['temp'])

predicted=84.238777, expected=82.125000 error=2.113777
predicted=82.643313, expected=81.750000 error=0.893313
predicted=82.338131, expected=86.375000 error=4.036869
predicted=85.951965, expected=87.285714 error=1.333749
predicted=86.044725, expected=85.000000 error=1.044725
predicted=84.132735, expected=80.000000 error=4.132735
predicted=80.714099, expected=83.857143 error=3.143044
predicted=84.295188, expected=84.000000 error=0.295188
predicted=84.469969, expected=84.875000 error=0.405031
predicted=84.078146, expected=86.750000 error=2.671854
predicted=86.634817, expected=84.125000 error=2.509817
predicted=82.922367, expected=82.250000 error=0.672367
predicted=82.798271, expected=81.625000 error=1.173271
predicted=82.693507, expected=82.625000 error=0.068507
predicted=83.126377, expected=84.000000 error=0.873623
predicted=84.331887, expected=84.625000 error=0.293113
predicted=84.060778, expected=84.750000 error=0.689222
predicted=83.978804, expected=83.500000 error=0.478804
predicted=

C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
C:\ProgramData\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an arra

In [16]:
new_data['temp'] = pred.flatten()

In [26]:
new_data = pd.DataFrame()

In [27]:
for i in aggregate_data.columns:
    print(i)
    pred = arima_ann(aggregate_data[i])
    new_data[i] = pred.flatten() 

dew_pt
Test MSE: 1.955


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, activation="tanh", input_dim=7, kernel_initializer="uniform")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="tanh", kernel_initializer="uniform")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(25, activation="tanh", kernel_initializer="uniform")`


error= 0.5253303620539147
hum
Test MSE: 31.495


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, activation="tanh", input_dim=7, kernel_initializer="uniform")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="tanh", kernel_initializer="uniform")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(25, activation="tanh", kernel_initializer="uniform")`


error= 1.7187518853128458
pressure
Test MSE: 0.002


...

error= 1.0437422445613014
vis
Test MSE: 0.051


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, activation="tanh", input_dim=7, kernel_initializer="uniform")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="tanh", kernel_initializer="uniform")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(25, activation="tanh", kernel_initializer="uniform")`


error= 0.052093222514077386
wind_spd
Test MSE: 31.319


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, activation="tanh", input_dim=7, kernel_initializer="uniform")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="tanh", kernel_initializer="uniform")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(25, activation="tanh", kernel_initializer="uniform")`


error= 11.794588696793841


In [33]:
new_data

,temp,dew_pt,hum,wind_spd,vis,pressure
0,84.585253,77.592904,78.354329,4.811010,1.619794,29.531831
1,84.663637,80.112681,83.854779,4.230057,1.616145,29.635678
2,86.119821,81.043827,78.562335,3.151507,1.655221,29.550405
3,85.286227,79.744264,81.270026,0.824502,1.751578,29.463505
4,82.523389,78.349781,89.225883,0.062748,1.666078,29.457236
5,82.052247,80.304473,94.221264,1.106363,1.390444,29.472943
6,82.123428,79.660274,91.867147,5.890757,1.487714,29.412029
7,83.145245,80.257681,77.482389,9.277407,1.512870,29.396533
8,85.328575,78.610376,77.463118,5.552695,1.503126,29.520275
9,84.341620,79.050142,77.931179,1.284431,1.686198,29.461280


In [32]:
new_data = new_data[['temp', 'dew_pt', 'hum', 'wind_spd', 'vis', 'pressure']]

In [29]:
import pickle as pkl

In [35]:
with open('gaussian_model.pkl','rb') as f:
    rain_model = pkl.load(f)

In [37]:
rain =rain_model.predict(new_data)

In [38]:
new_data['rain'] = rain

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
new_data[new_data['rain'] == 1]

,temp,dew_pt,hum,wind_spd,vis,pressure,rain
5,82.052247,80.304473,94.221264,1.106363,1.390444,29.472943,1
6,82.123428,79.660274,91.867147,5.890757,1.487714,29.412029,1
11,82.149860,79.204595,89.988561,0.892697,1.335031,29.441476,1
16,82.197928,79.504798,84.481257,-6.483935,1.713604,29.540063,1
17,80.001444,76.844814,95.895017,1.838231,1.499976,29.534363,1
18,81.214081,76.059568,85.531486,2.062945,1.436609,29.457950,1
43,81.373991,78.400730,86.163591,8.830555,1.439004,29.404392,1
63,80.863181,77.689643,89.731027,38.266444,1.512794,29.587414,1
64,82.942721,76.405641,75.611701,21.624807,1.168364,29.538287,1
